In [12]:
#pip install textblob
import pandas as pd
from textblob import TextBlob
from sklearn import preprocessing

In [2]:
data = pd.read_csv("dataset/all-data.csv",encoding="Windows-1252")

In [3]:
data.head()

,neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing ."
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...


Rename columns

In [4]:
data = data.rename(columns={"neutral":"Sentiment", data.columns[1]:"text"})

In [5]:
data.head()

,Sentiment,text
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...


Below, we describe a function to lowercase text, remove punctuation and symbols from words, and other useless chars/words.

In [6]:
from bs4 import BeautifulSoup
import re
def text_cleaning(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    
    text = text.replace("(<br/>)", "")
    text = text.replace('(<a).*(>).*(</a>)', '')
    text = text.replace('(&amp)', '')
    text = text.replace('(&gt)', '')
    text = text.replace('(&lt)', '')
    text = text.replace('(\xa0)', ' ')
    
    text = text.lower()
    return text
data['text'] = data['text'].apply(text_cleaning)

Remove duplicates.

In [7]:
data.drop_duplicates(subset=['text'],keep='first',inplace=True)

Added 3 columns:
1) text sentiment polarity by TextBlob library

2) Lenght of each news text

3) Count of words in each news

In [9]:
data['polarity'] = data['text'].map(lambda text: TextBlob(text).sentiment.polarity)
data['news_len'] = data['text'].astype(str).apply(len)
data['word_count'] = data['text'].apply(lambda x: len(str(x).split()))

In [11]:
data.head()

,Sentiment,text,polarity,news_len,word_count
0,neutral,technopolis plans to develop in stages an area...,0.083333,190,31
1,negative,the international electronic industry company ...,0.000000,228,36
2,positive,with the new production plant the company woul...,-0.064802,206,33
3,positive,according to the company 's updated strategy f...,0.000000,203,41
4,positive,financing of aspocomp 's growth aspocomp is ag...,0.500000,178,25


Label Encoded data.Sentiment column into numeric type 

In [15]:
le = preprocessing.LabelEncoder()
arr = le.fit_transform(data.Sentiment)

In [16]:
data["Sentiment"] = arr

In [22]:
data.head()

,Sentiment,text,polarity,news_len,word_count
0,1,technopolis plans to develop in stages an area...,0.083333,190,31
1,0,the international electronic industry company ...,0.000000,228,36
2,2,with the new production plant the company woul...,-0.064802,206,33
3,2,according to the company 's updated strategy f...,0.000000,203,41
4,2,financing of aspocomp 's growth aspocomp is ag...,0.500000,178,25


Converted text into vectors by using TFidfVectorizer model and now data is ready.

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(data['text'])